In [38]:
import pickle
import datetime as dt
import pandas as pd
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups#
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired

### Store and Preprocess Data

In [2]:
submissions1 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions1.json")
submissions2 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions2.json")
submissions3 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions3.json")
submissions4 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions4.json")
submissions5 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions5.json")
submissions6 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions6.json")
submissions7 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions7.json")

In [14]:
submissions['selftext'].to_csv("data_find")

In [26]:
submissions = pd.concat([submissions1, submissions2, submissions3, submissions4, submissions5, submissions6, submissions7], ignore_index=True)
print(submissions.size)
submissions = submissions.loc[(submissions['selftext'] != '') & (submissions['selftext'] != '[deleted]') & (submissions['selftext'] != '[removed]')]
submissions.dropna()
submissions.reset_index(drop=True, inplace=True)
mask = submissions['selftext'].str.len() >=1
submissions= submissions.loc[mask]
#posts['selftext'] = posts['title'].astype('str')
print(submissions.size)


29108532
17624970


In [28]:
submissions['created_utc'] = submissions['created_utc'].apply(lambda date: dt.datetime.fromtimestamp(date))
submissions = submissions.sort_values(by='created_utc', ascending=True)
years =set(submissions['created_utc'].dt.year.to_list())

### Calculate Topics by Years

In [29]:

if 0:
    print(year)
    sample_number = 3000
    posts = submissions[submissions['created_utc'].dt.year==year]


    print("sampling data")
    if(posts.shape[0] < 3000):
        sample_number = posts.shape[0]
    dfSample = posts['selftext'].sample(sample_number)
    data = dfSample.to_list()
    
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # fast but pretty accurate embeddings
    embeddings = embedding_model.encode(data, show_progress_bar=True)
    with open(str(year) + "embeddings.pkl", "wb") as fOut:
        pickle.dump({str(year)+'embeddings-only1': embeddings},fOut)

2016
sampling data


Batches: 100%|██████████| 94/94 [02:32<00:00,  1.62s/it]


2017
sampling data


Batches: 100%|██████████| 94/94 [02:22<00:00,  1.51s/it]


2018
sampling data


...

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.74it/s]


2014
sampling data


Batches: 100%|██████████| 94/94 [02:16<00:00,  1.45s/it]


2015
sampling data


Batches: 100%|██████████| 94/94 [02:17<00:00,  1.47s/it]


In [45]:
embeddings = {}
if 0:
    for year in years:
        with open(str(year) + "embeddings.pkl", "rb") as fIn:
            cache_data = pickle.load(fIn)
            print(cache_data)
            embeddings = cache_data[str(year)+'embeddings-only1']
        try:
            topicModel = BERTopic(verbose=True, embedding_model="all-MiniLM-L6-v2", representation_model=KeyBERTInspired()) 
            topics, probs = topicModel.fit_transform(data, embeddings)
        except:
            continue

{'2016embeddings-only1': array([[-2.8931147e-03, -1.0990363e-01,  6.6168144e-02, ...,
         9.0606883e-03,  6.5594348e-03, -6.5573463e-03],
       [-8.0260649e-02, -7.0326060e-02, -2.4151871e-02, ...,
         6.0399845e-03,  1.8606786e-02, -1.8828316e-02],
       [ 2.1699646e-02,  2.7360922e-05,  2.3694694e-02, ...,
        -6.5688647e-02, -6.5822583e-03, -2.2711031e-02],
       ...,
       [ 2.2890145e-02, -6.7461558e-02,  4.3521132e-03, ...,
        -5.5088274e-02, -1.9784969e-04, -2.8905377e-02],
       [ 6.7359857e-02, -8.5017018e-02, -5.8567179e-03, ...,
        -2.4620745e-02, -2.7051095e-02,  4.0021427e-02],
       [ 7.0983075e-02,  1.1113807e-01,  2.7779860e-02, ...,
        -3.0476460e-03, -9.4917715e-02,  1.9114187e-02]], dtype=float32)}


2023-11-20 17:49:05,908 - BERTopic - Reduced dimensionality
2023-11-20 17:49:06,006 - BERTopic - Clustered reduced embeddings


{'2017embeddings-only1': array([[-0.04409034, -0.09077199, -0.00893802, ...,  0.09062455,
        -0.08203623,  0.03371636],
       [ 0.01626818, -0.01278613,  0.01106399, ..., -0.00161768,
        -0.09539252, -0.02737349],
       [-0.00109263,  0.1089293 ,  0.02795395, ...,  0.03535517,
        -0.02318533, -0.0275852 ],
       ...,
       [-0.03249486,  0.0095879 ,  0.01783998, ..., -0.02543228,
        -0.04499633, -0.00801935],
       [ 0.08365713, -0.00994504,  0.01657425, ..., -0.02728137,
        -0.06881227,  0.00429414],
       [-0.02172235,  0.07705797,  0.04117063, ..., -0.01015177,
         0.00011748, -0.04086026]], dtype=float32)}


2023-11-20 17:49:39,013 - BERTopic - Reduced dimensionality
2023-11-20 17:49:39,109 - BERTopic - Clustered reduced embeddings


{'2018embeddings-only1': array([[ 0.08470776, -0.01665849,  0.07448334, ...,  0.00134327,
        -0.07450522, -0.04679236],
       [-0.01306077, -0.05099441, -0.00499397, ..., -0.03272002,
        -0.03330849,  0.0942742 ],
       [ 0.02537968, -0.03551613,  0.00499156, ..., -0.0937696 ,
        -0.03420861,  0.02941266],
       ...,
       [-0.12457715,  0.10605086,  0.04469365, ..., -0.02428617,
        -0.06795746, -0.01325911],
       [-0.01637382,  0.03666568,  0.04334334, ..., -0.08754168,
        -0.05433042,  0.08092182],
       [ 0.0135336 , -0.05795679,  0.12259805, ...,  0.05217906,
        -0.11027162, -0.04313593]], dtype=float32)}


...

{'2010embeddings-only1': array([[ 0.03388299, -0.06846802,  0.03841985, ...,  0.05455399,
        -0.00486806,  0.01869964],
       [-0.00330581,  0.00080769, -0.03523574, ...,  0.01124648,
         0.0504192 , -0.04749123],
       [ 0.04282231, -0.05474753,  0.00504814, ..., -0.01300391,
        -0.05396838, -0.00044658],
       ...,
       [ 0.05637384, -0.01025718, -0.00970191, ..., -0.01745078,
         0.04160207,  0.05710806],
       [ 0.02521606, -0.13671012, -0.05509052, ..., -0.02573266,
         0.08087453, -0.01877661],
       [ 0.11545262, -0.01121868, -0.07574148, ..., -0.02368018,
        -0.05258647, -0.01110857]], dtype=float32)}
{'2011embeddings-only1': array([[-0.00290896, -0.06732383,  0.03931407, ..., -0.13989131,
        -0.06387119,  0.04358809],
       [ 0.06019468, -0.0665935 ,  0.01171349, ..., -0.08103342,
        -0.1026013 ,  0.01119775],
       [ 0.0210888 , -0.05792421,  0.04812714, ..., -0.0562794 ,
        -0.091479  ,  0.03172365],
       ...,
       [ 

2023-11-20 17:52:28,696 - BERTopic - Reduced dimensionality
2023-11-20 17:52:28,794 - BERTopic - Clustered reduced embeddings


{'2013embeddings-only1': array([[ 0.00425331, -0.04146898,  0.05958175, ..., -0.02212209,
        -0.12164985,  0.02041013],
       [-0.10149891, -0.00981178,  0.03868761, ..., -0.12120683,
        -0.03522051, -0.02708027],
       [ 0.04843904, -0.02671784,  0.11511432, ..., -0.01883863,
         0.01414882,  0.07811869],
       [ 0.04380814, -0.00322477,  0.03955466, ..., -0.0174785 ,
        -0.09736925, -0.04361986],
       [ 0.00681317, -0.04165075,  0.02873897, ...,  0.01848924,
        -0.14966688,  0.03223858],
       [ 0.04567214, -0.0400871 , -0.05745888, ..., -0.02220304,
        -0.03625107,  0.06408536]], dtype=float32)}
{'2014embeddings-only1': array([[-0.08889083, -0.01860604, -0.03503602, ..., -0.02129542,
        -0.11735911,  0.00533552],
       [-0.01329427, -0.00386929,  0.03713627, ..., -0.07471276,
        -0.05758296, -0.0243179 ],
       [ 0.01099008, -0.00231486,  0.1632459 , ...,  0.01835486,
        -0.07468835, -0.11843091],
       ...,
       [ 0.01285856, 

2023-11-20 17:53:01,827 - BERTopic - Reduced dimensionality
2023-11-20 17:53:01,940 - BERTopic - Clustered reduced embeddings


{'2015embeddings-only1': array([[ 0.08098323, -0.05043215,  0.00016234, ..., -0.0117062 ,
         0.03085539,  0.0358707 ],
       [ 0.00774006,  0.03259801, -0.01544798, ..., -0.03104876,
        -0.08322182, -0.05940226],
       [ 0.06153182, -0.04914653,  0.06097639, ..., -0.05919405,
        -0.05664682,  0.01946494],
       ...,
       [ 0.02977203,  0.10366309,  0.03676306, ...,  0.01619382,
        -0.14754355, -0.03148817],
       [ 0.06079275, -0.07049439, -0.01995135, ..., -0.03904489,
        -0.07651559,  0.00427957],
       [-0.01641113,  0.05242947, -0.00965677, ..., -0.07338092,
        -0.06213282, -0.02573716]], dtype=float32)}


In [48]:
topic_model = BERTopic(verbose=True)
sample_df = submissions.sample(15000)
text = sample_df['selftext'].to_list()
dates = sample_df['created_utc'].to_list()

topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(text)

Batches: 100%|██████████| 469/469 [12:09<00:00,  1.55s/it]
2023-11-20 18:11:01,772 - BERTopic - Transformed documents to Embeddings
2023-11-20 18:11:31,915 - BERTopic - Reduced dimensionality
2023-11-20 18:11:32,852 - BERTopic - Clustered reduced embeddings


In [64]:
topics_over_time = topic_model.topics_over_time(text, dates, datetime_format="%b%M", nr_bins=10)

10it [00:06,  1.52it/s]


In [62]:
topic_nums= list(range(0, 11))

In [63]:
topic_model.visualize_topics_over_time(topics_over_time, topics=topic_nums)

In [65]:
fig = topic_model.visualize_topics_over_time(topics_over_time, topics=topic_nums)
fig.write_html("topics_over_time.html")